# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from sklearn.model_selection import train_test_split
import sys
from sklearn import metrics
from keras.models import load_model
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as skAUC
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, TensorBoard
import tensorflow as tf
from keras import backend as K
import time
import glob
from tqdm import tqdm
import os

In [ ]:
import pathlib
from keras import backend as K 
tf.config.experimental.list_physical_devices('GPU')
path = pathlib.Path.cwd()
path

In [ ]:
from kerastuner.tuners import RandomSearch ## pip3 install -U keras-tuner 

# Load data (please DIY)

In [ ]:
rinv = "0p3"
file_path = "LL-"+rinv+".h5"
hdf_file = path/file_path
hf = h5py.File(hdf_file, "r")

In [ ]:
X = hf["features"][:]
Y = hf["targets"][:]

In [ ]:
N = len(X)
Ntrain, Nval, Ntest = int(N/5*4),  int(N/10),  int(N/10)
Xim_train, Xim_val, Xim_test = X[:Ntrain], X[Ntrain:Nval+Ntrain], X[Nval+Ntrain:N]
yim_train, yim_val, yim_test = Y[:Ntrain], Y[Ntrain:Nval+Ntrain], Y[Nval+Ntrain:N]

# Define function

In [ ]:
class get_filter(tf.keras.layers.Layer):
    def __init__(self, units=1):
        super(get_filter, self).__init__()
        self.units = units
        
    def build(self, input_shape):  # Create the state of the layer (weights)
        wq_init = tf.random_normal_initializer()
#         wq_init = tf.zeros_initializer()
        self.ft = tf.Variable(initial_value=wq_init(shape=( input_shape[1], input_shape[2], self.units), dtype='float32'), trainable=True, name='Basisfinder_ft')
        self.shape = input_shape
#         super(Basisfilter, self).build()
        
    def call(self, inputs):  # Defines the computation from inputs to outputs
#         inputs = tf.expand_dims(inputs, axis=-1)
#         inputs = tf.tile(inputs,[1,1,1,self.units])
#         b = inputs - self.ft
        b = tf.math.abs(self.ft)
#         b = tf.math.sign(b)-0.6
#         b = (tf.math.sign(b) +1)/2
        return inputs*b


In [ ]:
def build_model(hp):
    inputs = tf.keras.Input(shape=(32,32))
  
    lr = hp.Choice('learning_rate', values=[i*1e-4 for i in range(10)])
#     batch = hp.Int('batch_size', min_value=32, max_value=512, step=16)
    convk1 = hp.Int('convk_1', min_value=1, max_value = 4, step=1)
    convp1 = hp.Int('convp_1', min_value=32, max_value =256, step=8, parent_values=[convk1])
    
    pool1 = hp.Int('pooling_1', min_value=2, max_value = 3, step=1, parent_values=[convk1, convp1])
    
    convk2 = hp.Int('convk_2', min_value=1, max_value = 4, step=1)
    convp2 = hp.Int('convp_2', min_value=32, max_value = 256, step=8, parent_values=[convk2])
    
    
    convk3 = hp.Int('convk_3', min_value=1, max_value = 4, step=1)
    convp3 = hp.Int('convp_3', min_value=32, max_value = 256, step=8, parent_values=[convk3])
    
    
    
    
    x = inputs
    x = tf.expand_dims(x,axis=-1)
    x = get_filter(1)(x)
    x = tf.keras.layers.Conv2D(convp1, kernel_size=(convk1, convk1), activation='relu', input_shape=(32,32,1))(x)
#     x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(pool1, pool1))(x)
    
#     x = tf.keras.layers.Conv2D(56, kernel_size=(3,3), activation='relu', input_shape=(64,64,32))(x)
    x = tf.keras.layers.Conv2D(convp2, kernel_size=(convk2, convk2), activation='relu')(x)
    
#     x = tf.keras.layers.Conv2D(56, kernel_size=(3,3), activation='relu', input_shape=(15,15,64))(x)
    x = tf.keras.layers.Conv2D(convp3, kernel_size=(convk3, convk3), activation='relu')(x)
    
    x = tf.keras.layers.Flatten()(x)
    
    dns1 = hp.Int('units_1', min_value=32, max_value=512, step=32, parent_values=[x.shape[1]])
    dp1 = hp.Choice('Dropout_1', values=[i/10 for i in range(10)], parent_values=[dns1])
    dns2 = hp.Int('units_2', min_value=32, max_value=256, step=32, parent_values=[dp1, dns1])
    dp2 = hp.Choice('Dropout_2', values=[i/10 for i in range(6)], parent_values=[dp1, dns1, dns2])
    x = tf.keras.layers.Dense(dns1, activation='relu')(x)
    x = tf.keras.layers.Dropout(dp1)(x)
    x = tf.keras.layers.Dense(dns2, activation='relu')(x)
    x = tf.keras.layers.Dropout(dp2)(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    modelCNN_basis = tf.keras.Model(inputs=inputs, outputs=x, name='CNN_basis')
    modelCNN_basis.compile(optimizer=keras.optimizers.Adam(lr) ,
                     loss="binary_crossentropy",
                     metrics=['accuracy'])
    print(hp.values)
    modelCNN_basis.summary()
#     print("Droup out :", dp1, " ", dp2)
#     print("lr :", lr)
                           
    return modelCNN_basis

# Tunner

In [ ]:
tuner = RandomSearch(build_model, objective='val_loss',max_trials=10, executions_per_trial=1,directory='./Keras_Tunner', project_name='CNN'+rinv)


In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(Xim_train, yim_train, epochs=5, batch_size=512, validation_data=(Xim_val, yim_val), verbose=0)

## Get best model

In [ ]:
models = tuner.get_best_models(num_models=1)

In [ ]:
HP = tuner.get_best_hyperparameters(num_trials=1)[0]


### Check model architecture

In [ ]:
HP.values

# Save model architecture

In [ ]:
import csv

In [ ]:
hpdic = HP.values 
# w = csv.writer(open('./Keras_Tunner/best_all_CNN_model_'+rinv+'.csv', "w"))
with open('./Keras_Tunner/best_all_CNN_model_'+rinv+'.csv', "w") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames= hpdic.keys())
    writer.writeheader()
    writer.writerows([hpdic])

# Load model architecture

In [ ]:
with open('./Keras_Tunner/best_all_CNN_model_'+rinv+'.csv', 'r') as f:
    reader = csv.reader(f)
    my_list = list(reader)
hpdirc = {my_list[0][i]:[my_list[1][i]] for i in range(len(my_list[0]))}


In [ ]:
def get_model(hp):
    inputs = tf.keras.Input(shape=(32,32))
  
    lr = float(hp['learning_rate'][0])
#     batch = hp.Int('batch_size', min_value=32, max_value=512, step=16)
    convk1 = int(hp['convk_1'][0])
    convp1 = int(hp['convp_1'][0])
    
    pool1 = int(hp['pooling_1'][0])
    
    convk2 = int(hp['convk_2'][0])
    convp2 = int(hp['convp_2'][0])
    
    
    convk3 = int(hp['convk_3'][0])
    convp3 = int(hp['convp_3'][0])
    
    dns1 = int(hp['units_1'][0])
    dp1 = float(hp['Dropout_1'][0])
    dns2 = int(hp['units_2'][0])
    dp2 = float(hp['Dropout_2'][0])
    
    
    x = inputs
    x = tf.expand_dims(x,axis=-1)
    x = get_filter(1)(x)
    x = tf.keras.layers.Conv2D(convp1, kernel_size=(convk1, convk1), activation='relu', input_shape=(32,32,1))(x)
#     x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(pool1, pool1))(x)
    
#     x = tf.keras.layers.Conv2D(56, kernel_size=(3,3), activation='relu', input_shape=(64,64,32))(x)
    x = tf.keras.layers.Conv2D(convp2, kernel_size=(convk2, convk2), activation='relu')(x)
    
#     x = tf.keras.layers.Conv2D(56, kernel_size=(3,3), activation='relu', input_shape=(15,15,64))(x)
    x = tf.keras.layers.Conv2D(convp3, kernel_size=(convk3, convk3), activation='relu')(x)
    
    x = tf.keras.layers.Flatten()(x)
    

    x = tf.keras.layers.Dense(dns1, activation='relu')(x)
    x = tf.keras.layers.Dropout(dp1)(x)
    x = tf.keras.layers.Dense(dns2, activation='relu')(x)
    x = tf.keras.layers.Dropout(dp2)(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    modelCNN_basis = tf.keras.Model(inputs=inputs, outputs=x, name='CNN_basis')
    modelCNN_basis.compile(optimizer=keras.optimizers.Adam(lr) ,
                     loss="binary_crossentropy",
                     metrics=['accuracy'])
    print(hp.values)
    modelCNN_basis.summary()
#     print("Droup out :", dp1, " ", dp2)
#     print("lr :", lr)
                           
    return modelCNN_basis

In [ ]:
modelCNN_basis = get_model(hpdirc)

# Train

In [ ]:
i = 3
opt = ["Adadelta", "SGD", "Adagrad", "Adam"]
optn = opt[i]
opts = {"Adadelta":[tf.keras.optimizers.Adadelta()], "SGD":[tf.keras.optimizers.SGD()], 
        "Adagrad":[tf.keras.optimizers.Adagrad()], "Adam":[tf.keras.optimizers.Adam(learning_rate = float(hpdirc['learning_rate'][0]))]}
optimizer =opts[optn][0]

# modelCNN.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
modelCNN_basis.compile(optimizer= optimizer, loss="binary_crossentropy", metrics=['accuracy'])

modelCNN_basis.summary()

In [ ]:
model_type = "CNN_basis"
save_dir = './Keras_Tunner/'
model_name = '%s_model.test.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

# lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)

# progress_bar = keras.callbacks.ProgbarLogger()

# csv_logger = keras.callbacks.CSVLogger(save_dir+'CNN'+rinv+'.csv')
# csv_logger = keras.callbacks.CSVLogger(save_dir+'CNN_'+rinv+'_'+optn+'_filter.csv')
csv_logger = keras.callbacks.CSVLogger(save_dir+'CNN_all_'+rinv+'_'+optn+'_filter.csv')


earlystop = tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            min_delta=1e-4,
                            patience=3,
                            verbose=1,
                            mode='min', baseline=None, ## 'min' 
                            restore_best_weights=True)
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                               patience=2, min_lr=0.00001)
callbacks = [checkpoint, csv_logger,  earlystop ]


In [ ]:
modelCNN_basis.fit(Xim_train, yim_train , validation_data=(Xim_val, yim_val), callbacks = callbacks, shuffle=True , epochs=400, verbose=0)

## Save model

In [ ]:
modelCNN_basis.save("./Keras_Tunner/CNN_all_"+rinv+"_"+optn+"_filter")
